# Pembangunan Model Fine-tuned YOLO untuk Deteksi Bangunan Wilayah DKI Jakarta pada Citra Satelit

# Melakukan Clone Arsitektur Darknet

Darknet diklon dari repositori github terkenal milik [AlexAB](https://github.com/AlexeyAB/darknet). Kemudian mengatur Makefile untuk mengaktifkan GPU dan OPENCV untuk selanjutnya membangun Darknet.


In [ ]:
# clone darknet repo
%cd .
!git clone https://github.com/AlexeyAB/darknet

/content
Cloning into 'darknet'...
remote: Enumerating objects: 15460, done.
remote: Total 15460 (delta 0), reused 0 (delta 0), pack-reused 15460
Receiving objects: 100% (15460/15460), 14.07 MiB | 7.52 MiB/s, done.
Resolving deltas: 100% (10385/10385), done.


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
# make darknet (build)
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

# Menyiapkan File dari Google Drive

**Catatan:** Membuat shortcut atau symbolic link antara `/content/drive/MyMyDrive/ColabNotebooks/ObjectDetection/Skripsi` dan `/mydrive` agar lebih efisien

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/drive')

/content
Mounted at /content/drive


In [ ]:
# this creates a symbolic link
!ln -s /content/drive/MyDrive/ColabNotebooks/ObjectDetection/Skripsi /mydrive

# Memindahkan Dataset ke Cloud VM 

Karena menggunakan CV, maka akan ada file berbeda untuk train dan test pada tiap fold, sehingga terdapat 5 file.

In [ ]:
import shutil

zipname = ["train_1", "train_2", "train_3", "train_4", "train_5", "test_1", 
           "test_2", "test_3", "test_4", "test_5"]

%cd /content/
for name in zipname:
  path = "/mydrive/data/split/"+ name + ".zip"
  shutil.copy(path, "/content/")

/content


In [ ]:
from subprocess import call

%cd /content/darknet/
for i in range(1, 6):
    call(["unzip", f'../train_{i}.zip', '-d', 'data/'])
    call(["unzip", f'../test_{i}.zip', '-d', 'data/'])

/content/darknet


# Konfigurasi File untuk Training


## i) File Cfg 

Semua hyperparameter pada pemodelan fine-tuned dibuat sama dengan pre-trained. Pada pemodelan ini, terdapat 3 jenis model sehingga file konfigurasi yang digunakan juga berbeda. Terdapat 3 file `cfg`, yakni `baseline.cfg`, `tuning_anc.cfg`, dan `tuning_per_wilayah.cfg`, masing-masing ditujukan untuk model dengan anchors box default, model modifikasi anchors box seluruh wilayah, dan model modifikasi anchors box per wilayah.


In [ ]:
import os

config_path = "/mydrive/configuration/cfg baseline"
for i in range(5):
  file = "baseline_" + str(i+1) + ".cfg"
  path = os.path.join(config_path, file)
  shutil.copy(path, "/content/darknet/cfg")

## ii) Files .names dan .data 

File `data` dibuat 5 file dengan nama yang berbeda, dikarenakan subset pada CV yang digunakan 5. Masing-masing file diberikan nama yang berurutan.

In [ ]:
for i in range(5):
  file = "train_" + str(i+1) + ".data"
  path = os.path.join("/mydrive/configuration/data_names", file)
  shutil.copy(path, "/content/darknet/data")
  
!cp /mydrive/configuration/train.names ./data

## iii) Generate File train.txt

Tidak seperti pada pemodelan pre-trained, di sini fungsi generate nama file train dan tes dibuat di dalam file. Tujuannya agar lebih efisien.


In [ ]:
def GenerateImageFile(folder):
  image_files = []
  path = os.path.join("data", folder)

  os.chdir(path)
  for filename in os.listdir(os.getcwd()):
      if filename.endswith(".jpg"):
          image_files.append(os.path.join(path, filename))

  os.chdir("..")
  filename = folder + ".txt"
  with open(filename, "w") as outfile:
      for image in image_files:
          outfile.write(image)
          outfile.write("\n")
      outfile.close()
      
  os.chdir("..")

In [ ]:
for i in range(1, 6):
    GenerateImageFile(f"train_{i}")
    GenerateImageFile(f"test_{i}")

# Training Object Detector YOLOv3

**TIP:** Proses training membutuhkan waktu beberapa jam (pada kasus ini sekitar 3 - 4 jam), tergantung dari seberapa besar iterasi yang digunakan pada file konfigurasi. Sedangkan Colab akan menghentikan running time jika tidak ada aktivitas. Agar kita bisa membiarkan Cloud VM berjalan sendirinya, bisa menggunakan cara di bawah.

Paste kode di bawah ke console, lalu enter.
```javascript
const addCodeBlock = () => {
    document.querySelector("colab-toolbar-button").click()
}
interval = setInterval(addCodeBlock, 1000*60*25) // 25 menit
```

In [ ]:
!./darknet detector train data/train_1.data cfg/baseline_1.cfg /mydrive/pretrained/final_join.weights -dont_show -map

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->

In [ ]:
!./darknet detector train data/train_2.data cfg/baseline_2.cfg /mydrive/pretrained/final_join.weights -dont_show -map

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->

In [ ]:
!./darknet detector train data/train_3.data cfg/baseline_3.cfg /mydrive/pretrained/final_join.weights -dont_show -map

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->

In [ ]:
!./darknet detector train data/train_4.data cfg/baseline_4.cfg /mydrive/pretrained/final_join.weights -dont_show -map

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->

In [ ]:
!./darknet detector train data/train_5.data cfg/baseline_5.cfg /mydrive/pretrained/final_join.weights -dont_show -map

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
 Prepare additional network for mAP calculation...
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->